In [14]:
import pandas as pd
from collections import Counter
from math import log, sqrt

In [10]:
def logodds(corpora_dic, bg_counter):
    """ It calculates the log odds ratio of term i's frequency between
    a target corpus and another corpus, with the prior information from
    a background corpus. Inputs are:

    - a dictionary of Counter objects (corpora of our interest)
    - a Counter objects (background corpus)

    Output is a dictionary of dictionaries. Each dictionary contains the log
    odds ratio of each word.

    """
    corp_size = dict([(c, sum(corpora_dic[c].values())) for c in corpora_dic])
    bg_size = sum(bg_counter.values())
    result = dict([(c, {}) for c in corpora_dic])

    for name, c in corpora_dic.items():
        for word in c:
            # if 10 > sum(1 for corpus in corpora_dic.values() if corpus[word]):
            #    continue

            fi = c[word]
            fj = sum(co.get(word, 0) for x, co in corpora_dic.items() if x != name)
            fbg = bg_counter[word]
            ni = corp_size[name]
            nj = sum(x for idx, x in corp_size.items() if idx != name)
            nbg = bg_size
            oddsratio = (
                log(fi + fbg)
                - log(ni + nbg - (fi + fbg))
                - log(fj + fbg)
                + log(nj + nbg - (fj + fbg))
            )
            std = 1.0 / (fi + fbg) + 1.0 / (fj + fbg)
            z = oddsratio / sqrt(std)
            result[name][word] = z

    return result

In [11]:
news_citations_df = pd.read_parquet("../data/intermediate/citation_analysis/news_citations.parquet")

In [39]:
gpt_cor = Counter(news_citations_df.query("model_family == 'gpt'").domain)
gemini_cor = Counter(news_citations_df.query("model_family == 'gemini'").domain)
perplexity_cor = Counter(news_citations_df.query("model_family == 'perplexity'").domain)

In [62]:
gpt_gemini_cor = Counter(news_citations_df.query("model_family == 'gpt' or model_family == 'gemini'").domain)
gpt_ppx_cor = Counter(news_citations_df.query("model_family == 'gpt' or model_family == 'perplexity'").domain)
gemini_ppx_cor = Counter(news_citations_df.query("model_family == 'gemini' or model_family == 'perplexity'").domain)
all_cor = Counter(news_citations_df.domain)

In [42]:
gpt_v_gemini_logodds = logodds(
    {
        "gpt": gpt_cor,
        "gemini": gemini_cor
    },
    gpt_gemini_cor
)
gpt_v_gemini_gpt_df = pd.DataFrame(gpt_v_gemini_logodds['gpt'].items(), columns=['domain', 'logodds']).sort_values(by='logodds')
gpt_v_gemini_gemini_df = pd.DataFrame(gpt_v_gemini_logodds['gemini'].items(), columns=['domain', 'logodds']).sort_values(by='logodds')

In [66]:
gpt_v_all_logodds = logodds(
    {
        "gpt": gpt_cor,
        "other": gemini_ppx_cor
    },
    all_cor
)
gpt_v_all_df = pd.DataFrame(gpt_v_all_logodds['gpt'].items(), columns=['domain', 'logodds']).sort_values(by='logodds', ascending=False)

gemini_v_all_logodds = logodds(
    {
        "gemini": gemini_cor,
        "other": gpt_ppx_cor
    },
    all_cor
)
gemini_v_all_df = pd.DataFrame(gemini_v_all_logodds['gemini'].items(), columns=['domain', 'logodds']).sort_values(by='logodds', ascending=False)

ppx_v_all_logodds = logodds(
    {
        "perplexity": perplexity_cor,
        "other": gpt_gemini_cor
    },
    all_cor
)
ppx_v_all_df = pd.DataFrame(ppx_v_all_logodds['perplexity'].items(), columns=['domain', 'logodds']).sort_values(by='logodds', ascending=False)

In [69]:
gpt_v_all_df.head(10)

,domain,logodds
6,reuters.com,34.089759
10,apnews.com,22.219095
2,ft.com,19.726681
7,axios.com,18.426373
33,time.com,11.430856
13,as.com,10.625586
21,theatlantic.com,8.638108
26,elpais.com,7.665638
9,theguardian.com,6.030787
91,biomedcentral.com,4.636705


In [70]:
gemini_v_all_df.head(10)

,domain,logodds
17,indiatimes.com,8.382392
9,livemint.com,6.624592
29,aljazeera.com,6.510891
52,thehindu.com,6.168107
103,dailymail.co.uk,5.310493
48,independent.co.uk,3.792984
227,history.com,3.584977
19,ndtv.com,3.581090
21,hindustantimes.com,3.490315
8,forbes.com,3.452091


In [71]:
ppx_v_all_df.head(10)

,domain,logodds
669,bbc.com,9.673403
659,nytimes.com,8.405172
162,yahoo.com,7.698068
18,espn.com,6.469246
678,cnn.com,6.069012
702,cnbc.com,5.833757
31,sohu.com,5.803378
41,163.com,5.765142
86,economictimes.com,5.528315
21,techcrunch.com,5.418761


In [45]:
gpt_v_ppx_logodds = logodds(
    {
        "gpt": gpt_cor,
        "perplexity": perplexity_cor
    },
    gpt_ppx_cor
)
gpt_v_ppx_gpt_df = pd.DataFrame(gpt_v_ppx_logodds['gpt'].items(), columns=['domain', 'logodds']).sort_values(by='logodds')
gpt_v_ppx_ppx_df = pd.DataFrame(gpt_v_ppx_logodds['perplexity'].items(), columns=['domain', 'logodds']).sort_values(by='logodds')

In [56]:
gemini_v_ppx_logodds = logodds(
    {
        "gemini": gemini_cor,
        "perplexity": perplexity_cor
    },
    gemini_ppx_cor
)
gemini_v_ppx_gemini_df = pd.DataFrame(gemini_v_ppx_logodds['gemini'].items(), columns=['domain', 'logodds']).sort_values(by='logodds')
gemini_v_ppx_ppx_df = pd.DataFrame(gemini_v_ppx_logodds['perplexity'].items(), columns=['domain', 'logodds']).sort_values(by='logodds')

In [49]:
gpt_v_ppx_gpt_df.tail(20)

,domain,logodds
50,cnews.fr,2.030712
184,scrippsnews.com,2.167943
28,windowscentral.com,2.359643
549,ipsnews.net,2.428256
239,thehindu.com,2.684732
137,history.com,2.684732
69,knowyourmeme.com,2.823125
428,asahi.com,2.918822
82,makeuseof.com,3.967477
22,lemonde.fr,4.045449


In [50]:
gpt_v_gemini_gpt_df.tail(20)

,domain,logodds
242,heise.de,1.948724
211,wired.com,2.005368
55,venturebeat.com,2.060471
167,yna.co.kr,2.132453
32,ria.ru,2.217670
46,espn.com,2.267678
91,biomedcentral.com,2.547465
8,usnews.com,2.839777
38,rbc.ru,3.212417
9,theguardian.com,3.316248


In [58]:
gpt_v_gemini_gemini_df.tail(20)

,domain,logodds
113,clevelandclinic.org,3.524316
191,bmj.com,3.561140
19,ndtv.com,3.590433
54,pbs.org,3.660867
10,healthline.com,3.733176
103,dailymail.co.uk,3.741642
27,zdnet.com,3.803250
68,news.cn,3.846910
52,thehindu.com,3.855945
80,medicalnewstoday.com,3.878160


In [57]:
gemini_v_ppx_gemini_df.tail(20)

,domain,logodds
440,arabnews.com,2.462762
531,reliefweb.int,2.469801
212,infobae.com,2.474071
99,cbr.com,2.481169
5,techradar.com,2.937298
8,forbes.com,2.940150
327,dexerto.com,3.054151
21,hindustantimes.com,3.086943
55,apnews.com,3.156450
95,fool.com,3.178988


In [59]:
gpt_v_ppx_ppx_df.tail(20)

,domain,logodds
680,wsj.com,3.105683
97,globo.com,3.112063
42,cbsnews.com,3.116201
10,cbr.com,3.171313
63,pbs.org,3.180960
763,npr.org,3.239880
762,nbcnews.com,3.450802
21,techcrunch.com,3.625572
31,sohu.com,3.673999
6,accuweather.com,3.682321


In [61]:
gemini_v_ppx_ppx_df.tail(20)

,domain,logodds
705,bbc.co.uk,2.528966
118,usatoday.com,2.573616
163,nasdaq.com,2.603600
113,businessinsider.com,2.687103
763,npr.org,2.758326
76,euronews.com,2.816032
162,yahoo.com,2.835815
510,dw.com,2.894585
97,globo.com,2.949127
86,economictimes.com,3.139418
